# 🎯 Configuração de Trigger: Table Update (Atualização de Tabela)

Este notebook demonstra como configurar um **Lakeflow Job** com trigger do tipo **Table Update** para processar automaticamente a camada silver quando a tabela bronze é atualizada.

## 📋 Objetivo

Criar um job que executa automaticamente o pipeline **bronze_to_silver** (deduplicação + join) sempre que a tabela `claims` na camada bronze for atualizada.

## 🔄 Fluxo do Trigger Table Update

1. **Monitoramento**: O Lakeflow Jobs monitora a tabela `${catalog}.${schema_bronze}.claims`
2. **Detecção**: Quando novos dados são inseridos/atualizados na tabela
3. **Disparo**: O job é automaticamente executado
4. **Processamento**: 
   - Task 1: Deduplica claims (`01_deduplicate_claims.py`)
   - Task 2: Cria tabela enriched (`02_join_tables_create.sql`)
   - Task 3: Calcula métricas (`03_join_tables_metrics.sql`)
   - Task 4: Mostra amostra (`04_join_tables_sample.sql`)

## 📝 Pré-requisitos

- ✅ Catálogo e schemas criados
- ✅ Tabela `claims` na camada bronze já existe
- ✅ Scripts bronze_to_silver disponíveis


## 🛠️ Passo 1: Criar o Job no Lakeflow Jobs

1. Acesse **Lakeflow Jobs** no menu lateral do Databricks
2. Clique em **Create Job**
3. Configure o nome: `bronze_to_silver_table_update`
4. Adicione **4 Tasks** sequenciais


## 📂 Passo 2: Configurar as Tasks

### Task 1: Deduplicação (Python)

**Task Name:** `deduplicate_claims`

**Task Type:** `Python script`

**Source:** `/Workspace/.../aula_05/bronze_to_silver/01_deduplicate_claims.py`

**Parameters:**
- `catalog` = `smart_claims_dev`
- `schema_bronze` = `01_bronze`
- `schema_silver` = `02_silver`

---

### Task 2: Criar Tabela Enriched (SQL)

**Task Name:** `create_enriched_table`

**Task Type:** `SQL script`

**Source:** `/Workspace/.../aula_05/bronze_to_silver/02_join_tables_create.sql`

**Parameters:**
- `catalog` = `smart_claims_dev`
- `schema_bronze` = `01_bronze`
- `schema_silver` = `02_silver`

**Depends on:** `deduplicate_claims`

---

### Task 3: Métricas (SQL)

**Task Name:** `calculate_metrics`

**Task Type:** `SQL script`

**Source:** `/Workspace/.../aula_05/bronze_to_silver/03_join_tables_metrics.sql`

**Parameters:**
- `catalog` = `smart_claims_dev`
- `schema_silver` = `02_silver`

**Depends on:** `create_enriched_table`

---

### Task 4: Amostra (SQL)

**Task Name:** `show_sample`

**Task Type:** `SQL script`

**Source:** `/Workspace/.../aula_05/bronze_to_silver/04_join_tables_sample.sql`

**Parameters:**
- `catalog` = `smart_claims_dev`
- `schema_silver` = `02_silver`

**Depends on:** `create_enriched_table` (pode executar em paralelo com Task 3)


## ⚡ Passo 3: Configurar o Trigger Table Update

### No painel de configuração do Job:

1. Vá para a seção **Triggers**
2. Clique em **Add trigger**
3. Selecione **Table update**

### Configurações do Trigger:

**Trigger Type:** `Table update`

**Table to monitor:** 
```
smart_claims_dev.01_bronze.claims
```

**Update type:**
- ✅ **INSERT** - Dispara quando novos dados são inseridos
- ✅ **UPDATE** - Dispara quando dados são atualizados
- ✅ **DELETE** - Dispara quando dados são deletados
- (Recomendado: marcar apenas INSERT para este caso)

**Wait for completion:** 
- ✅ Marque esta opção para evitar execuções concorrentes

**Min time between triggers (seconds):**
- Exemplo: `300` (5 minutos)
- Evita múltiplas execuções se houver muitas atualizações rápidas


## 🧪 Passo 4: Testar o Trigger

### Simular atualização da tabela bronze:

Execute o código abaixo para inserir novos dados na tabela `claims` e disparar o trigger:


In [ ]:
# Simular atualização da tabela bronze
# Este código insere novos dados na tabela claims para disparar o trigger

from pyspark.sql import Row
from datetime import datetime

catalog = "smart_claims_dev"
schema_bronze = "01_bronze"
table = f"{catalog}.{schema_bronze}.claims"

# Criar dados de teste
test_data = [
    Row(
        claim_no=f"TEST_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        policy_no="POL_TEST_001",
        claim_date="2024-01-15",
        months_as_customer=12,
        injury=0,
        property=1000,
        vehicle=5000,
        total=6000,
        collision_type="Rear-end",
        number_of_vehicles_involved=2,
        age=35,
        insured_relationship="Self",
        license_issue_date="2010-01-01",
        date="2024-01-15",
        hour=14,
        type="Collision",
        severity="Medium",
        number_of_witnesses=1,
        suspicious_activity="No"
    )
]

# Criar DataFrame e inserir na tabela
test_df = spark.createDataFrame(test_data)
test_df.write.mode("append").saveAsTable(table)

print(f"✅ Dados de teste inseridos na tabela: {table}")
print(f"📊 O job deve ser disparado automaticamente em alguns segundos!")
print(f"🔍 Verifique o status do job em: Lakeflow Jobs > bronze_to_silver_table_update")


## 📊 Passo 5: Monitorar Execuções

### Verificar execuções do job:

1. Acesse **Lakeflow Jobs** > `bronze_to_silver_table_update`
2. Veja o histórico de execuções na aba **Runs**
3. Cada execução mostra:
   - **Trigger:** Table update
   - **Tabela monitorada:** `smart_claims_dev.01_bronze.claims`
   - **Status de cada task:** Success, Failed, ou Running
   - **Tempo total de execução**

### Verificar resultados:

Execute o código abaixo para verificar se os dados foram processados corretamente:


In [ ]:
# Verificar se os dados foram processados na camada silver

catalog = "smart_claims_dev"
schema_silver = "02_silver"

# Verificar tabela deduplicada
dedup_table = f"{catalog}.{schema_silver}.claims_dedup"
dedup_count = spark.table(dedup_table).count()
print(f"📊 Total de registros em {dedup_table}: {dedup_count}")

# Verificar tabela enriched
enriched_table = f"{catalog}.{schema_silver}.claims_enriched"
enriched_count = spark.table(enriched_table).count()
print(f"📊 Total de registros em {enriched_table}: {enriched_count}")

# Mostrar amostra
print("\n🔍 Amostra dos dados enriched:")
spark.table(enriched_table).select(
    "claim_no", "customer_name", "borough", "MAKE", "MODEL", "claim_total", "severity"
).show(5, truncate=False)


## ✅ Vantagens do Trigger Table Update

- ✅ **Pipeline automatizado**: Bronze → Silver executa automaticamente
- ✅ **Sincronização**: Dados silver sempre atualizados quando bronze muda
- ✅ **Eficiência**: Só executa quando há mudanças reais
- ✅ **Confiabilidade**: Evita dados desatualizados na camada silver

## ⚠️ Considerações Importantes

- O trigger monitora a tabela continuamente
- Múltiplas atualizações podem disparar múltiplas execuções
- Use `Min time between triggers` para evitar sobrecarga
- Configure dependências corretas entre tasks
- Considere usar **Delta Live Tables (DLT)** para pipelines mais complexos
